https://iexcloud.io/console/

In [ ]:
# Установка toads
# !pip install --upgrade git+https://github.com/ivanrychkov/toads

In [2]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
# from toads.image import Img
import time
import datetime


pd.set_option('max_columns', None)

Документация:

https://iexcloud.io/docs/api/#historical-prices

In [1]:
class Stocks:
    """Класс для работы с API iexcloud."""
    valid_times = {'max', '5y', '2y', '1y', 'ytd', '6m', '3m', '1m', '1mm', '5d', '5dm', 'date', 'dynamic'}
    default_symbol = 'aapl'
    
    def __init__(self, symbol = default_symbol, token: str = None):
        """Создаёт контекст для работы с акцией."""
        if not token:
            print('Using sandbox token.')
            self.token = 'Tpk_ef2b6bae433d4597a222690ed9fb967c'
        else:
            self.token = token
            
        self.symbol = symbol
    
    def __str__(self):
        return f'iexcloud API context for {self.symbol.upper()} stocks'
    
    __repr__ = __str__
    
    @staticmethod
    def format_time(string, **kws):
        """Принимает строку и превращает её в дату в необходимом формате."""
        return pd.to_datetime(string, **kws).date().isoformat().replace('-', '', -1)
        
    def get_chart(self, time_range = None, include_today = True, **kwargs):
        """Делает http-запрос к API iexcloud и возвращает его результат.
        
        time_range:
        max: All available data up to 15 years - Historically adjusted market-wide data
        5y: Five years - Historically adjusted market-wide data
        2y: Two years - Historically adjusted market-wide data
        1y: One year - Historically adjusted market-wide data
        ytd: Year-to-date - Historically adjusted market-wide data
        6m: Six months - Historically adjusted market-wide data
        3m: Three months - Historically adjusted market-wide data
        1m: One month (default) - Historically adjusted market-wide data
        1mm: One month - Historically adjusted market-wide data in 30 minute intervals
        5d: Five Days - Historically adjusted market-wide data by day.
        5dm: Five Days - Historically adjusted market-wide data in 10 minute intervals
        date: Specific date - If used with the query parameter chartByDay, then this returns historical OHLCV data for that date. Otherwise, it returns data by minute for a specified date, if available. Date format YYYYMMDD. Currently supporting trailing 30 calendar days of minute bar data.
        dynamic: One day - Will return 1d or 1m data depending on the day or week and time of day. Intraday per minute data is only returned during market hours.

        full doc at:
        https://iexcloud.io/docs/api/#historical-prices
        """
        # Адреса
        base_url = 'https://sandbox.iexapis.com/stable'
        endpoint = f'/stock/{self.symbol}/chart/'

        #  Разбираемся с параметрами
        params = {'token': self.token if self.token else 'Tpk_ef2b6bae433d4597a222690ed9fb967c',
                  'includeToday': 'true' if include_today else 'false'}
        
        if not time_range:
            time_range = 'dynamic'
        # Дата или диапазон
        if time_range not in Stocks.valid_times:
            params['exactDate'] = ''.join(Stocks.format_time(time_range))
        else:
            params['range'] = time_range

        # Если есть ключевые слова, дополним ими словарь
        params.update(kwargs)

        # Делаем запрос
        request = requests.get(base_url + endpoint, params=params)

        # Проверяем код запроса
        assert request.status_code == 200, f'Status code: {request.status_code}. Check your query parameters.'
        return request

    def get_chart_df(self, time_range = '1m', **kwargs):
        """Делает http-запрос к iexcloud и формирует из него датафрейм."""
        # Делаем запрос
        return Stocks.make_df(self.get_chart(time_range=time_range, **kwargs).json())
        
    @staticmethod
    def make_df(json_request):
        """Формирует датафрейм из JSON."""
        def intraday(json_request):
            """Формирует датафрейм с поминутными данными."""
            data = pd.DataFrame(json_request)
            data.loc[:, 'datetime'] = pd.to_datetime(data.date + ' ' + data.minute)
            data = data.drop(['minute', 'date', 'label'], axis=1).set_index('datetime')
            return data
        
        def daily(json_request):
            """Формирует датафрейм с подневными данными."""
            data = pd.DataFrame(json_request)
            data.loc[:, 'datetime'] = pd.to_datetime(data.date)
            data = data.drop(['minute', 'date', 'label'], axis=1, errors='ignore').set_index('datetime')
            return data
        
        def dynamic(json_request):
            return pd.DataFrame(json_request['data'])
        
        # Для пустых запросов - пустой датафрейм
        if len(json_request) == 0:
            return pd.DataFrame()
        
        # Сначала пробуем поминутно
        try:
            if 'minute' in json_request[0].keys():
                return intraday(json_request)
        except:
            pass
        
        # Пробуем динамическое время
        try:
            if 'range' in json_request.keys():
                return dynamic(json_request)
        except:
            pass
        
        return daily(json_request)
    
    def get_n_last_dates(self, n=1, last_date = None, preproc_func = lambda x: x, eager = True):
        """Возвращает данные за последние n дней.
        
        eager - возвращает готовый датафрейм или генератор с ленивым вычислением для поочерёдного перебора.
        preproc_func - функция, предобрабатывающая каждый датафрейм. Должна возвращать датафрейм.
        """
        generator = (self.get_chart_df(Stocks.format_time(date))
                     for date
                     in pd.date_range(end=last_date if last_date else datetime.date.today().isoformat(), periods=n))
        return pd.concat(list(generator)) if eager else generator

In [25]:
# Создаём контекст, в котором укажем акцию
stocks = Stocks('yndx')
stocks

Using sandbox token.


iexcloud API context for YNDX stocks

In [11]:
# Вызовем датафрейм за последние 5 дней по 10 минут
stocks.get_chart_df(time_range='5dm').head()

,open,high,low,close,average,volume,notional,numberOfTrades,marketOpen,marketHigh,marketLow,marketClose,marketAverage,marketVolume,marketNotional,marketNumberOfTrades
datetime,,,,,,,,,,,,,,,,
2020-11-09 09:30:00,66.020,68.650,66.720,68.34,66.742305,16786,1.120145e+06,138,66.34,68.01,66.21,68.200,66.726387,153704,1.046260e+07,843
2020-11-09 09:40:00,65.770,69.030,66.350,67.82,68.676173,8608,5.545930e+05,106,68.27,69.00,66.28,67.980,68.442846,134080,9.032710e+06,831
2020-11-09 09:50:00,67.999,67.211,65.531,66.70,67.135700,17382,1.128847e+06,93,68.85,67.05,67.86,66.668,67.685710,124962,8.064278e+06,733
2020-11-09 10:00:00,66.210,68.310,66.780,66.94,68.363086,2199,1.437772e+05,22,68.29,69.07,68.50,68.790,66.737063,52963,3.433107e+06,428
2020-11-09 10:10:00,67.250,68.000,65.820,66.00,67.857576,3228,2.140169e+05,24,68.98,68.00,68.81,68.260,66.458640,79630,5.306327e+06,635


In [6]:
# Вызовем датафрейм за один день поминутно
stocks.get_chart_df(time_range='20201112').head()

,high,low,average,volume,notional,numberOfTrades,marketHigh,marketLow,marketAverage,marketVolume,marketNotional,marketNumberOfTrades,open,close,marketOpen,marketClose,changeOverTime,marketChangeOverTime
datetime,,,,,,,,,,,,,,,,,,
2020-11-12 09:30:00,79.62,77.77,80.730,2141,164592.20,18,78.59,79.70,80.192,126336,1.005535e+07,163,80.68,79.80,78.00,79.490,0.000000,0.000000
2020-11-12 09:31:00,78.43,77.35,77.530,77,5973.30,1,78.33,79.16,80.322,5881,4.463784e+05,47,80.42,77.43,77.43,80.042,0.000094,0.001158
2020-11-12 09:32:00,77.36,80.68,80.436,440,34912.32,4,80.78,77.70,80.000,23013,1.779449e+06,115,79.59,77.84,77.70,78.550,-0.000910,0.002698
2020-11-12 09:33:00,78.60,79.84,81.300,1778,139562.25,14,78.94,79.10,81.333,31711,2.473771e+06,220,80.42,77.82,80.11,79.900,0.003746,0.005538
2020-11-12 09:34:00,80.81,80.92,80.687,1326,106314.92,13,81.26,77.98,79.300,18864,1.461082e+06,123,78.55,80.73,81.15,78.980,0.006736,0.008240


In [7]:
# Вызовем датафреймы за несколько дат, включая выходные
stocks.get_n_last_dates(5).info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1170 entries, 2020-11-11 09:30:00 to 2020-11-13 15:59:00
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   high                  1110 non-null   float64
 1   low                   1110 non-null   float64
 2   average               1110 non-null   float64
 3   volume                1170 non-null   int64  
 4   notional              1170 non-null   float64
 5   numberOfTrades        1170 non-null   int64  
 6   marketHigh            1170 non-null   float64
 7   marketLow             1170 non-null   float64
 8   marketAverage         1170 non-null   float64
 9   marketVolume          1170 non-null   int64  
 10  marketNotional        1170 non-null   float64
 11  marketNumberOfTrades  1170 non-null   int64  
 12  open                  1110 non-null   float64
 13  close                 1110 non-null   float64
 14  marketOpen            1170 non-null 

In [8]:
# Функция для нескольких дат поддерживает ленивое вычисление (скачивает данные по запросу, а не сразу)
for day, data in enumerate(stocks.get_n_last_dates(365, eager=False)):
    # Возьмём только первые 3 дня
    if day == 3:
        break
    print(f'\nday {day + 1}\n')
    print(data.shape)


day 1

(0, 0)

day 2

(390, 18)

day 3

(390, 18)


In [ ]:
yndx = stocks()

---